In [9]:
import pyarrow.parquet as pq

table = pq.read_table(parquet_file)

df = table.to_pandas()

json_data = df.to_json(orient='records')

with open('../../data/raw/train-00000-of-00001-dcec52ec7fe8275d.json', 'w') as f:
    f.write(json_data)


In [5]:
import pandas as pd
data = pd.read_json("../../data/raw/cocktail_recipes.json")
data

,title,ingredients,directions,misc,source,ner
0,Mimosa,"[90 ml Champagne, 90 ml Orange juice]",[1 Champagne: Slowly pour the Champagne into y...,[With a 50/50 mix of Champagne and Orange Juic...,cocktailclub,"[champagne, orange juice]"
1,Green Coconut,"[1 Bars spoon Matcha tea powder, 4 Bar spoons ...","[1 Prepare: With your Fruit Zester, peel off a...",[This creamy and sweet detox mocktail is a hea...,cocktailclub,[]


In [3]:
%pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   --------- ------------------------------ 122.9/542.0 kB 3.5 MB/s eta 0:00:01
   ----------------- ---------------------- 235.5/542.0 kB 2.4 MB/s eta 0:00:01
   -------------------------- ------------- 358.4/542.0 kB 2.5 MB/s eta 0:00:01
   ----------------------------------- ---- 481.3/542.0 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/172.0 kB ? eta -:--:--
   ---------------------------------------- 172.0/172.0 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--
   ----------------------- ---------------- 215.0/371.6 kB 6.6 MB/s eta 0:00:01
   ---------------------------------------  368.6/371.6 kB 4.6 MB/s eta 0:00

In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../../data/raw/cocktail_recipes.json", split="train")

In [5]:
dataset

Dataset({
    features: ['directions', 'ner', 'misc', 'source', 'ingredients', 'title'],
    num_rows: 2
})

In [6]:
dataset[0]

{'directions': ['1 Champagne: Slowly pour the Champagne into your flute glass. 90 ml Champagne',
  '2 Orange juice: Pour the orange juice through the Champagne. 90 ml Orange juice',
  '3 Stir gently: Using your barspoon, mix the two ingredients gently. '],
 'ner': ['champagne', 'orange juice'],
 'misc': ['With a 50/50 mix of Champagne and Orange Juice, the Mimosa is a fruitier version of The Bellini. Often served at Weddings, receptions and other celebratory moments.   ',
  ' As well as the Champagne, keep the Orange Juice cold (and fresh if possible)'],
 'source': 'cocktailclub',
 'ingredients': ['90 ml Champagne', '90 ml Orange juice'],
 'title': 'Mimosa'}

In [7]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
      --------------------------------------- 0.3/12.8 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.8 MB 3.5 MB/s eta 0:00:04
     --- ------------------------------------ 1.3/12.8 MB 5.0 MB/s eta 0:00:03
     ----- ---------------------------------- 1.9/12.8 MB 6.3 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 7.1 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 7.6 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.8 MB 8.1 MB/s eta 0:00:02
     ------------- -------------------------- 4.4/12.8 MB 8.5 MB/s eta 0:00:01
     -------------- ------------------------- 4.8/12.8 MB 8.7 MB/s eta 0:00:01
     ---------------- ----------------------- 5.4/12.8 MB 8.5 MB/

In [10]:
import spacy

nlp = spacy.load("en_core_web_sm")

for index, item in data.iterrows():
    text_combined = ""
    for text in item['directions']:
        text_combined += text + " "
    for text in item['misc']:
        text_combined += text + " "

    doc = nlp(text_combined)
    
    data.loc[index, 'tokens_size'] = len(doc)
    index += 1

data

,title,ingredients,directions,misc,source,ner,tokens_size
0,Mimosa,"[90 ml Champagne, 90 ml Orange juice]",[1 Champagne: Slowly pour the Champagne into y...,[With a 50/50 mix of Champagne and Orange Juic...,cocktailclub,"[champagne, orange juice]",95.0
1,Green Coconut,"[1 Bars spoon Matcha tea powder, 4 Bar spoons ...","[1 Prepare: With your Fruit Zester, peel off a...",[This creamy and sweet detox mocktail is a hea...,cocktailclub,[],241.0


In [3]:
import sqlite3
from datasets import Dataset
con = sqlite3.connect("../../data/raw/cocktail_recipe.db")

dataset.to_sql(con = con, name = 'cocktail_recipe', if_exists='replace', index=False)

Creating SQL from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 10.13ba/s]


2